In [ ]:
# Step 1: Install and Import Libraries

!pip install pywavelets brisque scikit-image
!pip install --upgrade imageio
!pip install opencv-contrib-python

import os
import numpy as np
import cv2
import pywt
from imageio import v2 as imageio
from scipy.ndimage import sobel
from tqdm import trange, tqdm
import matplotlib.pyplot as plt
from brisque import BRISQUE
from skimage.metrics import structural_similarity as ssim
from google.colab import files
import zipfile
import shutil
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

print("✅ All libraries installed and imported.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 3.2 MB/s eta 0:00:00
  Created wheel for libsvm-official: filename=libsvm_official-3.36.0-cp312-cp312-linux_x86_64.whl size=124636 sha256=edb8fbf1a966fc4b706b29db5b5afc638949888248080e3f2e215ee38705a2db
  Stored in directory: /root/.cache/pip/wheels/df/65/4b/c3cdece6e5fa7eebef116be2d5a309f7ac50c90183cbe12c92
Successfully built libsvm-official
✅ All libraries installed and imported.


In [ ]:
# Step 2: Upload Your Input Image

uploaded = files.upload()

if not uploaded:
    raise Exception("No file was uploaded. Please run the cell again.")

INPUT_IMAGE_PATH = list(uploaded.keys())[0]
print(f"✅ Successfully uploaded '{INPUT_IMAGE_PATH}'")

Saving ArtRoom.png to ArtRoom.png
✅ Successfully uploaded 'ArtRoom.png'


In [ ]:
# Step 3: Define Experiment Configuration

OUTPUT_DIR = 'Seam_Carving_Optimal_2D_Results'
REDUCTION_WIDTH_PERCENT = 0.3 # Reduce width by 30%
REDUCTION_HEIGHT_PERCENT = 0.2 # Reduce height by 20%

# DWT Params
WAVELET_TYPE = 'db2'
DWT_LEVEL = 3

# Fusion Params
SALIENCY_WEIGHT = 0.5
DWT_DETAIL_WEIGHT = 1.0 - SALIENCY_WEIGHT

# Protection Mask Params
PROTECTION_THRESHOLD = 0.3
PROTECTION_COST = 1e9

print(f"✅ Configuration set. Output will be saved to '{OUTPUT_DIR}'")

✅ Configuration set. Output will be saved to 'Seam_Carving_Optimal_2D_Results'


In [ ]:
# Step 4: Define All Functions AND Run the Experiment (Corrected 2D Algorithm)

# Energy Map Functions
def get_sobel_importance_map(image):
    """(Baseline) Calculates the standard Sobel gradient energy map."""
    if image.ndim == 3 and image.shape[2] == 3:
        gray = cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_BGR2GRAY)
    else:
        gray = image.astype(np.uint8)
    dx = sobel(gray, axis=1, mode='constant')
    dy = sobel(gray, axis=0, mode='constant')
    return (np.abs(dx) + np.abs(dy)).astype(np.float32)

def get_saliency_map_only(image):
    """(Strategy 1) Generates a saliency map to find the main subject."""
    img_uint8 = image.astype(np.uint8)
    if img_uint8.ndim == 2:
        img_uint8 = cv2.cvtColor(img_uint8, cv2.COLOR_GRAY2BGR)
    if img_uint8.shape[2] == 4:
        img_uint8 = cv2.cvtColor(img_uint8, cv2.COLOR_BGRA2BGR)
    saliency = cv2.saliency.StaticSaliencySpectralResidual_create()
    (success, saliencyMap) = saliency.computeSaliency(img_uint8)
    if saliencyMap is None:
        saliencyMap = np.zeros(image.shape[:2])
    saliencyMap = (saliencyMap * 255).astype(np.float32)
    return saliencyMap

def get_dwt_detail_map_only(image):
    """(Strategy 2) Generates an importance map using ONLY DWT Detail bands."""
    if image.ndim == 3 and image.shape[2] == 3:
        gray = cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_BGR2GRAY) / 255.0
    else:
        gray = image.astype(np.uint8) / 255.0
    coeffs = pywt.wavedec2(gray, wavelet=WAVELET_TYPE, level=DWT_LEVEL)
    rec_coeffs = [np.zeros_like(coeffs[0])]
    for i in range(1, len(coeffs)):
        rec_coeffs.append(tuple(np.abs(c) for c in coeffs[i]))
    importance_map = pywt.waverec2(rec_coeffs, wavelet=WAVELET_TYPE)
    h, w = gray.shape
    importance_map = cv2.resize(importance_map, (w, h))
    if np.max(importance_map) > 0:
        importance_map = (importance_map / np.max(importance_map)) * 255.0
    return importance_map.astype(np.float32)

def get_dwt_saliency_fusion_map(image):
    """(Strategy 3) Fuses DWT Details with Saliency."""
    dwt_map = get_dwt_detail_map_only(image)
    sal_map = get_saliency_map_only(image)
    dwt_norm = dwt_map / (np.max(dwt_map) + 1e-6)
    sal_norm = sal_map / (np.max(sal_map) + 1e-6)
    fused_map = (DWT_DETAIL_WEIGHT * dwt_norm) + (SALIENCY_WEIGHT * sal_norm)
    fused_map = (fused_map / (np.max(fused_map) + 1e-6)) * 255.0
    return fused_map.astype(np.float32)

def get_dwt_saliency_protection_map(image):
    """(Strategy 4) Uses DWT Details + Saliency as a hard "Protection Mask"."""
    importance_map = get_dwt_detail_map_only(image)
    sal_map = get_saliency_map_only(image)
    sal_norm = sal_map / (np.max(sal_map) + 1e-6)
    protection_mask = (sal_norm > PROTECTION_THRESHOLD)
    importance_map[protection_mask] += PROTECTION_COST
    return importance_map.astype(np.float32)

# Core Seam Carving Logic
def get_minimum_seam(importance_map):
    """Finds the single optimal vertical seam path."""
    r, c = importance_map.shape
    M = importance_map.copy()
    backtrack = np.zeros_like(M, dtype=np.int32)
    for i in range(1, r):
        for j in range(c):
            left = M[i - 1, j - 1] if j > 0 else np.inf
            middle = M[i - 1, j]
            right = M[i - 1, j + 1] if j < c - 1 else np.inf
            min_energy = min(left, middle, right)
            if min_energy == np.inf: min_energy = 0
            M[i, j] += min_energy
            if min_energy == left:
                backtrack[i, j] = j - 1
            elif min_energy == middle:
                backtrack[i, j] = j
            else:
                backtrack[i, j] = j + 1
    seam = np.zeros(r, dtype=np.int32)
    j = np.argmin(M[-1])
    for i in reversed(range(r)):
        seam[i] = j
        j = backtrack[i, j]
    return seam, M

def carve_seam(img, seam):
    """Removes a given vertical seam from an image."""
    r, c, _ = img.shape
    mask = np.ones((r, c), dtype=bool)
    mask[np.arange(r), seam] = False
    return img[mask].reshape((r, c - 1, 3))

def _get_seam_costs(img, num_seams_to_remove, importance_map_func):
    """
    Helper function: Sequentially removes seams and records the
    energy cost of each seam *at the time of its removal*.
    This pre-computation is used to build the transport map.
    """
    temp_img = img.copy()
    costs = []
    for _ in range(num_seams_to_remove):
        importance_map = importance_map_func(temp_img)
        seam, _ = get_minimum_seam(importance_map)

        # Get the total energy of this seam
        cost = importance_map[np.arange(temp_img.shape[0]), seam].sum()
        costs.append(cost)

        # Remove the seam for the next iteration
        temp_img = carve_seam(temp_img, seam)
    return costs

def seam_carving_resize_optimal(img, target_h, target_w, importance_map_func):
    """
    Resizes an image to target (h, w) using the optimal 2D order.
    This is the CORRECT implementation from the paper.
    """
    img_resized = img.copy()
    h, w, _ = img.shape
    num_rows_to_remove = h - target_h
    num_cols_to_remove = w - target_w

    func_name = importance_map_func.__name__.replace("get_", "").replace("_importance_map", "").replace("_only", "")
    print(f"\nCalculating optimal path for: {func_name}")

    # 1. Pre-calculate costs for all vertical and horizontal seams
    vertical_costs = _get_seam_costs(img_resized, num_cols_to_remove, importance_map_func)

    img_rot = np.rot90(img_resized, 1, (0, 1))
    horizontal_costs = _get_seam_costs(img_rot, num_rows_to_remove, importance_map_func)

    # 2. Build the transport map T
    T = np.zeros((num_rows_to_remove + 1, num_cols_to_remove + 1))
    choices = np.zeros_like(T, dtype=int)

    for i in range(1, num_rows_to_remove + 1):
        T[i, 0] = T[i - 1, 0] + horizontal_costs[i - 1]
        choices[i, 0] = 0
    for j in range(1, num_cols_to_remove + 1):
        T[0, j] = T[0, j - 1] + vertical_costs[j - 1]
        choices[0, j] = 1

    for i in range(1, num_rows_to_remove + 1):
        for j in range(1, num_cols_to_remove + 1):
            cost_from_top = T[i - 1, j] + horizontal_costs[i - 1]
            cost_from_left = T[i, j - 1] + vertical_costs[j - 1]

            if cost_from_top < cost_from_left:
                T[i, j] = cost_from_top
                choices[i, j] = 0
            else:
                T[i, j] = cost_from_left
                choices[i, j] = 1

    # 3. Backtrack to find the optimal sequence of operations
    path = []
    i, j = num_rows_to_remove, num_cols_to_remove
    while i > 0 or j > 0:
        if choices[i, j] == 0:
            path.append('h')
            i -= 1
        else:
            path.append('v')
            j -= 1
    path.reverse() # Path is built backwards, so reverse it

    # 4. Apply the optimal sequence of removals
    seams_removed_energy = []
    pbar = tqdm(path, desc=f"Carving optimal path ({func_name})")
    for direction in pbar:
        importance_map = importance_map_func(img_resized)

        if direction == 'v':
            seam, _ = get_minimum_seam(importance_map)
            seam_energy = np.mean(importance_map[np.arange(img_resized.shape[0]), seam])
            img_resized = carve_seam(img_resized, seam)
        else:
            img_rot = np.rot90(img_resized, 1, (0, 1))
            importance_map_rot = importance_map_func(img_rot)
            seam, _ = get_minimum_seam(importance_map_rot)
            seam_energy = np.mean(importance_map_rot[np.arange(img_rot.shape[0]), seam])
            img_carved_rot = carve_seam(img_rot, seam)
            img_resized = np.rot90(img_carved_rot, 3, (0, 1))

        seams_removed_energy.append(seam_energy)

    return img_resized, seams_removed_energy

# Metric Calculation Functions
def calculate_brisque(image):
    """Calculates BRISQUE score. Lower is better (more 'natural')."""
    try:
        if image.ndim == 3 and image.shape[2] == 3:
            gray_img = cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_RGB2GRAY)
        else:
            gray_img = image.astype(np.uint8)
        brisque_obj = BRISQUE(url=False)
        return brisque_obj.score(gray_img)
    except Exception as e:
        return np.nan

def align_and_compare_ssim(map1, map2):
    """Helper to resize map2 to map1's size and compute SSIM."""
    h, w = map1.shape
    map2_aligned = cv2.resize(map2, (w, h), interpolation=cv2.INTER_LINEAR)
    map1_norm = (map1 - np.min(map1)) / (np.max(map1) - np.min(map1) + 1e-6)
    map2_norm = (map2_aligned - np.min(map2_aligned)) / (np.max(map2_aligned) - np.min(map2_aligned) + 1e-6)
    score, _ = ssim(map1_norm, map2_norm, full=True, data_range=1.0)
    return score

def calculate_edge_ssim(original, resized):
    """Compares the *structure* of the edge maps. Higher is better."""
    edge_map_original = get_sobel_importance_map(original)
    edge_map_resized = get_sobel_importance_map(resized)
    return align_and_compare_ssim(edge_map_original, edge_map_resized)

def calculate_saliency_ssim(original, resized):
    """Compares the *structure* of the saliency maps. Higher is better."""
    saliency_map_original = get_saliency_map_only(original)
    saliency_map_resized = get_saliency_map_only(resized)
    return align_and_compare_ssim(saliency_map_original, saliency_map_resized)

# Plotting Functions
def plot_importance_maps(img, maps_dict, save_path):
    print("Plotting 1: Importance Map Comparison")
    num_maps = len(maps_dict) + 1
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes.flat[0].imshow(img.astype(np.uint8)); axes.flat[0].set_title("(a) Original")
    for i, (name, map_data) in enumerate(maps_dict.items()):
        ax = axes.flat[i+1]
        im = ax.imshow(map_data, cmap='plasma'); fig.colorbar(im, ax=ax, label="Energy Level")
        ax.set_title(f"({chr(98+i)}) {name}")
    for ax in axes.flat: ax.axis('off')
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, "1_importance_maps.png"))
    plt.close()

def plot_path_maps(path_maps_dict, save_path):
    print("Plotting 2: Path Map Comparison")
    num_maps = len(path_maps_dict)
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    for i, (name, map_data) in enumerate(path_maps_dict.items()):
        ax = axes.flat[i]
        im = ax.imshow(map_data, cmap='viridis'); fig.colorbar(im, ax=ax, label="Cumulative Energy")
        ax.set_title(f"{name} Path Map")
        ax.axis('off')
    for i in range(num_maps, len(axes.flat)): axes.flat[i].axis('off')
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, "2_path_maps.png"))
    plt.close()

def plot_seam_overlays(img, seams_dict, save_path):
    print("Plotting 3: Seam Overlay Comparison")
    num_maps = len(seams_dict)
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    colors = [(255, 0, 0), (0, 255, 255), (0, 255, 0), (255, 0, 255), (255, 255, 0)]
    for i, (name, seam) in enumerate(seams_dict.items()):
        ax = axes.flat[i]
        img_with_seam = img.copy()
        for r in range(img.shape[0]):
            cv2.circle(img_with_seam, (seam[r], r), 1, colors[i % len(colors)], 2)
        ax.imshow(img_with_seam.astype(np.uint8)); ax.set_title(f"First Seam ({name})")
        ax.axis('off')
    for i in range(num_maps, len(axes.flat)): axes.flat[i].axis('off')
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, "3_seam_overlays.png"))
    plt.close()

def plot_resized_results(img, resized_images_dict, save_path):
    print("Plotting 4: Resizing Strategy Comparison")
    num_maps = len(resized_images_dict) + 1
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes.flat[0].imshow(img.astype(np.uint8))
    axes.flat[0].set_title(f"(a) Original ({img.shape[1]}x{img.shape[0]})")

    def save_img_with_dims(img_data, name_prefix):
        h, w, _ = img_data.shape
        filename = os.path.join(save_path, f"resized_{name_prefix}_{w}x{h}.png")
        imageio.imwrite(filename, img_data.astype(np.uint8))

    save_img_with_dims(img, "original")

    for i, (name, img_data) in enumerate(resized_images_dict.items()):
        ax = axes.flat[i+1]
        ax.imshow(img_data.astype(np.uint8))
        ax.set_title(f"({chr(98+i)}) {name} ({img_data.shape[1]}x{img_data.shape[0]})")
        save_img_with_dims(img_data, name)

    for ax in axes.flat: ax.axis('off')
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, "4_strategy_comparison.png"))
    plt.close()

def plot_energy_preservation(seam_energies_dict, save_path):
    print("Plotting 5: Energy Preservation Comparison")
    plt.figure(figsize=(12, 7))
    for name, energies in seam_energies_dict.items():
        avg_seam_energy = np.cumsum(energies) / (np.arange(len(energies)) + 1)
        plt.plot(avg_seam_energy, label=name, marker='o', markersize=4)
    plt.title("Energy of Removed Seams (Evaluated on their own maps)")
    plt.xlabel(f"Number of Seams Removed (Total {len(next(iter(seam_energies_dict.values())))})")
    plt.ylabel("Cumulative Average Energy of Seams (Lower is Better)")
    plt.legend()
    plt.grid(True, linestyle='--')
    plt.savefig(os.path.join(save_path, "5_energy_preservation.png"))
    plt.close()

# Main Experiment Functions
def main_experiment():
    print(f"--- Starting Seam Carving Full Comparison ---")
    print(f"Input Image: {INPUT_IMAGE_PATH}")
    print(f"Output Directory: {OUTPUT_DIR}")

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    img_original = imageio.imread(INPUT_IMAGE_PATH).astype(np.float32)
    if img_original.ndim == 2:
        img_original = cv2.cvtColor(img_original, cv2.COLOR_GRAY2RGB)
    if img_original.shape[2] == 4:
        img_original = img_original[:, :, :3]

    h, w, _ = img_original.shape
    target_w = w - int(w * REDUCTION_WIDTH_PERCENT)
    target_h = h - int(h * REDUCTION_HEIGHT_PERCENT)

    print(f"Original Image Dimensions: {w}x{h}")
    print(f"Target Dimensions: {target_w}x{target_h}\n")

    strategies = {
        "Sobel (Baseline)": get_sobel_importance_map,
        "Saliency_Only": get_saliency_map_only,
        "DWT_Detail_Only": get_dwt_detail_map_only,
        "DWT+Saliency_Fusion": get_dwt_saliency_fusion_map,
        "DWT+Saliency_Protection": get_dwt_saliency_protection_map
    }

    metrics_log = []
    importance_maps = {}
    path_maps = {}
    first_seams = {}
    resized_images = {}
    seam_energies = {}

    metrics_log.append("Seam Carving Comparison Metrics Log")
    metrics_log.append("="*40)
    metrics_log.append(f"Base Image: {INPUT_IMAGE_PATH} ({w}x{h})")
    metrics_log.append(f"Target Dimensions: {target_w}x{target_h}\n")

    print("\n--- Running All Resizing Strategies ---")

    orig_brisque = calculate_brisque(img_original)
    metrics_log.append("[Original Image]")
    metrics_log.append(f"  Dimensions: {img_original.shape[1]}x{img_original.shape[0]}")
    metrics_log.append(f"  BRISQUE: {orig_brisque:.4f}\n")

    for name, func in strategies.items():
        # Get initial maps and seams
        imp_map = func(img_original)
        seam, path_map = get_minimum_seam(imp_map)
        importance_maps[name] = imp_map
        path_maps[name] = path_map
        first_seams[name] = seam

        # Resizing
        resized_img, energies = seam_carving_resize_optimal(img_original, target_h, target_w, func)
        resized_images[name] = resized_img
        seam_energies[name] = energies

        # Calculating all the metrics
        brisque = calculate_brisque(resized_img)
        edge_ssim = calculate_edge_ssim(img_original, resized_img)
        sal_ssim = calculate_saliency_ssim(img_original, resized_img)

        metrics_log.append(f"[{name}]")
        metrics_log.append(f"  Dimensions: {resized_img.shape[1]}x{resized_img.shape[0]}")
        metrics_log.append(f"  BRISQUE:     {brisque:.4f} (Lower is better)")
        metrics_log.append(f"  Edge SSIM:   {edge_ssim:.4f} (Higher is better)")
        metrics_log.append(f"  Saliency SSIM: {sal_ssim:.4f} (Higher is better)\n")

    # Generating all comparison plots
    print("\n--- Generating All Comparison Plots ---")
    plot_importance_maps(img_original, importance_maps, OUTPUT_DIR)
    plot_path_maps(path_maps, OUTPUT_DIR)
    plot_seam_overlays(img_original, first_seams, OUTPUT_DIR)
    plot_resized_results(img_original, resized_images, OUTPUT_DIR)
    plot_energy_preservation(seam_energies, OUTPUT_DIR)

    # Saving all metrics and results
    log_path = os.path.join(OUTPUT_DIR, "comparison_metrics.txt")
    with open(log_path, 'w') as f:
        f.write("\n".join(metrics_log))

    print(f"\n✅ All results and metrics saved to '{OUTPUT_DIR}'")

    # Displaying final results
    print("\n--- FINAL METRICS SUMMARY (also saved to .txt) ---")
    df = pd.DataFrame(columns=['BRISQUE (Low)', 'Edge SSIM (High)', 'Saliency SSIM (High)'])
    for log_line in metrics_log:
        if log_line.startswith('['):
            current_method = log_line.strip('[]')
        if "BRISQUE:" in log_line:
            df.loc[current_method, 'BRISQUE (Low)'] = float(log_line.split(':')[1].split('(')[0].strip())
        if "Edge SSIM:" in log_line:
            df.loc[current_method, 'Edge SSIM (High)'] = float(log_line.split(':')[1].split('(')[0].strip())
        if "Saliency SSIM:" in log_line:
            df.loc[current_method, 'Saliency SSIM (High)'] = float(log_line.split(':')[1].split('(')[0].strip())

    print(df.to_markdown(floatfmt=".4f"))

# Run the experiment
main_experiment()

--- Starting Seam Carving Full Comparison ---
Input Image: ArtRoom.png
Output Directory: Seam_Carving_Optimal_2D_Results
Original Image Dimensions: 1024x813
Target Dimensions: 717x651


--- Running All Resizing Strategies ---

Calculating optimal path for: sobel


Carving optimal path (sobel): 100%|██████████| 469/469 [09:15<00:00,  1.19s/it]



Calculating optimal path for: saliency_map


Carving optimal path (saliency_map): 100%|██████████| 469/469 [09:09<00:00,  1.17s/it]



Calculating optimal path for: dwt_detail_map


...


Calculating optimal path for: dwt_saliency_fusion_map


Carving optimal path (dwt_saliency_fusion_map): 100%|██████████| 469/469 [09:51<00:00,  1.26s/it]



Calculating optimal path for: dwt_saliency_protection_map


Carving optimal path (dwt_saliency_protection_map): 100%|██████████| 469/469 [09:43<00:00,  1.24s/it]



--- Generating All Comparison Plots ---
Plotting 1: Importance Map Comparison
Plotting 2: Path Map Comparison
Plotting 3: Seam Overlay Comparison
Plotting 4: Resizing Strategy Comparison
Plotting 5: Energy Preservation Comparison

✅ All results and metrics saved to 'Seam_Carving_Optimal_2D_Results'

--- FINAL METRICS SUMMARY (also saved to .txt) ---
|                         |   BRISQUE (Low) |   Edge SSIM (High) |   Saliency SSIM (High) |
|:------------------------|----------------:|-------------------:|-----------------------:|
| Original Image          |             nan |           nan      |               nan      |
| Sobel (Baseline)        |             nan |             0.0081 |                 0.7865 |
| Saliency_Only           |             nan |             0.0057 |                 0.6885 |
| DWT_Detail_Only         |             nan |             0.0037 |                 0.6561 |
| DWT+Saliency_Fusion     |             nan |             0.0047 |                 0.5921 |
| D

In [ ]:
# Step 5: Zip and Download All Results

try:
    shutil.make_archive(f"{OUTPUT_DIR}", 'zip', OUTPUT_DIR)
    print(f"\nAll results saved and zipped to {OUTPUT_DIR}.zip")
    print("Downloading file...")
    files.download(f"{OUTPUT_DIR}.zip")
except FileNotFoundError:
    print(f"Error: Output directory '{OUTPUT_DIR}' not found. Did Step 4 run correctly?")


All results saved and zipped to Seam_Carving_Optimal_2D_Results.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>